# BERT pretrained model 제작

가장 대표적인 BERT만 하더라도 정식 모델은 340M나 되는 파라미터 사이즈를 자랑합니다. 

이들을 수십 GB나 되는 코퍼스를 토대로 학습시키는 것은 최고 성능의 GPU를 가지고도 수일 내지 수 주일의 시간이 걸립니다.  
아마도 여러분 대부분의 학습환경에서 이를 수행하는 것은 가능한 일이 아닐 것입니다.

그래서 오늘은 일반적인 10M 정도의 작은 파라미터 사이즈의 BERT 모델을 만들어, 수백 MB 수준의 코퍼스 기반으로 pretrain 을 진행해 보도록 하겠습니다.

학습에 사용할 코퍼스 데이터는 다음과 같습니다.  
https://d3s0tskafalll9.cloudfront.net/media/documents/kowiki.txt.zip

### 목차
- 1. 들어가며
- 2. Tokenizer 준비
- 3. 데이터 전처리 (1) MASK 생성
- 4. 데이터 전처리 (2) NSP pair 생성
- 5. 데이터 전처리 (3) 데이터셋 완성
- 6. BERT 모델 구현
- 7. pretrain 진행
- 8. 프로젝트 : mini BERT 만들기

## Tokenizer 준비

BERT등의 pretrained model이 나오게 되었을 즈음 자연어처리 분야의 또 다른 중요한 흐름 중 하나는 BPE 등의 subword 기반의 토크나이징 기법이 주요한 방법론으로 굳어졌다는 점입니다. 

GPT의 BPE, BERT의 WordPiece 모델 등의 성공이 더욱 사람들에게 subword 기반의 토크나이저에 대한 확신을 주었습니다.

오늘 우리는 SentencePiece 기반의 토크나이저를 준비하는 것으로 BERT pretrain 과정을 시작할 것입니다.

In [1]:
# imports
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow.keras.backend as K

import os
import re
import math
import numpy as np
import pandas as pd
import random
import collections
import json
import shutil
import zipfile
import copy
from datetime import datetime

import matplotlib.pyplot as plt
import sentencepiece as spm
from tqdm.notebook import tqdm

random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

# tf version 및 gpu 확인
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
print(tf.test.gpu_device_name())

2.10.0
[]



준비해 둔 한글 나무위키 코퍼스로부터 32000의 vocab_size를 갖는 sentencepiece 모델을 생성해 보겠습니다.

BERT에 사용되는 [MASK], [SEP], [CLS] 등의 주요 특수문자가 vocab에 포함되어야 함에 주의해 주세요.  
아래와 같이 모델을 생성하게 되면 약 30분 정도가 소요될 것입니다. 

그래서 아래 코드는 보기만하고 넘어가고 미리 만들어 놓은 파일을 아래 처럼 따라하여 사용하겠습니다.

In [2]:
# 실행하길 원한다면 """ 를 지워주세요.
"""
import sentencepiece as spm
import os
corpus_file = os.getenv('HOME')+'/aiffel/bert_pretrain/data/kowiki.txt'
prefix = 'ko_32000'
vocab_size = 32000

spm.SentencePieceTrainer.train(
    f"--input={corpus_file} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰
"""
print("완료=3")   # 완료메시지가 출력될 때까지 아무 출력내용이 없더라도 기다려 주세요.

완료=3


In [3]:
data_dir = os.getenv('HOME')+'/aiffel/bert_pretrain/data'
model_dir = os.getenv('HOME')+'/aiffel/bert_pretrain/models'

# vocab loading
vocab = spm.SentencePieceProcessor()
vocab.load(f"{model_dir}/ko_32000.model")

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

토크나이저가 잘 만들어졌는지 확인해 봅시다. 어떤 토큰이 만들어졌는지, 토크나이징 결과가 어떻게 나오는지 살펴볼까요?

In [4]:
# 특수 token 7개를 제외한 나머지 tokens 들
vocab_list = []
for id in range(7, len(vocab)):
    if not vocab.is_unknown(id):
        vocab_list.append(vocab.id_to_piece(id))
print(vocab_list)

NameError: name 'vocab' is not defined

In [ ]:
# [CLS], tokens a, [SEP], tokens b, [SEP] 형태의 token 생성
string_a = "추적추적 비가 내리는 날이었어 그날은 왠지 손님이 많아 첫 번에 삼십 전 둘째번 오십 전 오랜만에 받아보는 십 전짜리 백통화 서푼에"
string_b = "손바닥 위엔 기쁨의 눈물이 흘러 컬컬한 목에 모주 한잔을 적셔 몇 달 포 전부터 콜록거리는 아내 생각에 그토록 먹고 싶다던"
tokens_org = ["[CLS]"] + vocab.encode_as_pieces(string_a) + ["[SEP]"] + vocab.encode_as_pieces(string_b) + ["[SEP]"]
print(tokens_org)

토크나이저가 잘 작동하나요? 방금 우리는 SentencePiece 모델을 이용해 간단한 BERT의 Masked Language Model 학습용 데이터를 하나 생성해 보았습니다.

다음 절부터 본격적으로 데이터 전처리 과정에 돌입하겠습니다.